# Sample

## Transform

In [ ]:
# # Transformasi
# numerical_transformer = StandardScaler()
# categorical_transformer = Pipeline([
#     ('onehot', OneHotEncoder(handle_unknown = 'ignore', sparse_output = True, max_categories = 50)), 
#     # ('svd', TruncatedSVD(n_components = 100))  # Mengurangi dimensi fitur kategori
# ])
# ordinal_transformer = OrdinalEncoder(categories = oridnal_cat, handle_unknown = 'use_encoded_value', unknown_value = -1)

# # Column Transformer
# prep_stage_2 = ColumnTransformer(
#     transformers = [
#         ("num", numerical_transformer, numeric_cols), 
#         ("cat", categorical_transformer, one_hot_cols), 
#         ("ord", ordinal_transformer, ordinal_cols)
#     ], remainder = "passthrough")

In [ ]:
# # Transform data menggunakan fit_transform
# model_df = prep_stage_2.fit_transform(model_df)

# # Get name column from fit_transform
# model_df = pd.DataFrame(model_df, columns = prep_stage_2.get_feature_names_out())

# # Hilangkan prefix (misalnya, "num__", "cat__", "out__")
# clean_columns = [col.split("__", 1)[-1] for col in model_df.columns]
# model_df.columns = clean_columns

In [ ]:
# # Menampilkan total null pada setiap kolom
# null_columns = model_df.isnull().sum()[model_df.isnull().sum() > 0]
# print(f'Total null columns: {null_columns} \n')

In [ ]:
# # check data type after transform
# model_df.info()

In [ ]:
# # change object after transform
# model_df = convert_object_columns_to_numeric(model_df)
# model_df.info()

In [ ]:
# model_df.head()

## Base Model

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report, confusion_matrix

# # Define pipeline
# base_model_pipe = Pipeline([
#     ("model", RandomForestClassifier(n_estimators = 100, random_state = 42, class_weight = "balanced"))
# ])

# # Cross-validation scores with multiple metrics
# val_score = ["accuracy", "precision", "recall", "f1", "roc_auc"]
# cv_results = cross_validate(base_model_pipe, X_train, y_train, cv = 5, scoring = val_score)
# cv_results

In [ ]:
# print(f"Cross-validation Accuracy: {cv_results['test_accuracy'].mean():.4f}")
# print(f"Cross-validation Precision: {cv_results['test_precision'].mean():.4f}")
# print(f"Cross-validation Recall: {cv_results['test_recall'].mean():.4f}")
# print(f"Cross-validation F1-score: {cv_results['test_f1'].mean():.4f}")
# print(f"Cross-validation AUC-ROC: {cv_results['test_roc_auc'].mean():.4f} \n")

In [ ]:
# # Fit the pipeline
# base_model_pipe.fit(X_train, y_train)

# # Predict
# y_pred = base_model_pipe.predict(X_test)

# # Evaluasi Model
# print(classification_report(y_test, y_pred))

#### Evaluasi Model

In [ ]:
# from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

# # Evaluate Model
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc_score = roc_auc_score(y_test, y_pred)

# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1-Score: {f1:.4f}")
# print(f"AUC-ROC Score: {auc_score:.4f}")

## Hyperparameter

### Optimal CV

In [ ]:
# from sklearn.model_selection import cross_val_score
# import numpy as np

# cv_values = [2, 3, 5, 10]  # Coba berbagai nilai cv
# scores = []

# for cv in cv_values:
#     cv_score = cross_val_score(base_model_pipe, 
#                                X_train, 
#                                y_train, 
#                                cv = cv, 
#                                scoring = 'accuracy')
    
#     mean_score = np.mean(cv_score)
#     std_score = np.std(cv_score)
    
#     scores.append((cv, mean_score, std_score))
#     print(f"Cross Validation: {cv} -> Mean: {mean_score:.4f}, Std: {std_score:.4f}")

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Coba berbagai nilai k
cv_values = [3, 5, 7, 10]
results = {}

for k in cv_values:
    cv_strategy = StratifiedKFold(n_splits = k, shuffle = True, random_state = 42)
    scores = cross_val_score(base_model_pipe, X, y, cv = cv_strategy, scoring = 'accuracy')
    results[k] = (np.mean(scores), np.std(scores))  # Simpan rata-rata dan standar deviasi

# Cetak hasil
for k, (mean_score, std_dev) in results.items():
    print(f"CV={k}: Mean Accuracy={mean_score:.4f}, Std Dev={std_dev:.4f}")

In [ ]:
import matplotlib.pyplot as plt

k_values = list(results.keys())
mean_scores = [results[k][0] for k in k_values]
std_devs = [results[k][1] for k in k_values]

plt.figure(figsize=(8,5))
plt.errorbar(k_values, mean_scores, yerr=std_devs, fmt='o-', capsize=5, label='Mean Accuracy ± Std Dev')
plt.xlabel('Number of CV Folds (k)')
plt.ylabel('Accuracy')
plt.title('Finding Optimal CV for Cross Validation')
plt.legend()
plt.grid()
plt.show()


### Randomsearch

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# import matplotlib.pyplot as plt

# n_iter_values = [10, 20, 50, 100]  # Uji berbagai nilai
# scores = []

# param_grid = {
#     "model__n_estimators": [50, 100, 200, 500],
#     "model__max_depth": [None, 10, 20, 30],
#     "model__min_samples_split": [2, 5, 10, 20],
#     "model__min_samples_leaf": [1, 2, 5, 10],
#     "model__max_features": ["sqrt", "log2", None],
#     "model__bootstrap": [True, False],
#     "model__class_weight": ["balanced", "balanced_subsample", None]
# }

# for n in n_iter_values:
#     random_search = RandomizedSearchCV(
#         base_model_pipe, 
#         param_grid, 
#         n_iter = n, 
#         cv = 3, 
#         scoring = "recall", 
#         n_jobs = -1, 
#         random_state = 42
#     )

#     random_search.fit(X_train, y_train)
#     scores.append(random_search.best_score_)

# plt.plot(n_iter_values, scores, marker='o')
# plt.xlabel("n_iter")
# plt.ylabel("Best CV Score")
# plt.title("Evaluasi n_iter di RandomizedSearchCV")
# plt.show()

In [ ]:
# random_search = RandomizedSearchCV(
#     base_model_pipe,    # Model pipeline
#     param_distributions = param_grid,   # Gunakan param_grid yang sama
#     n_iter = 20,    # Batasi jumlah kombinasi yang diuji (misal 20)
#     cv = 5,     # Cross-validation 5-fold
#     scoring = ["precision", "recall", "f1"], 
#     refit = "recall",
#     n_jobs = -1,    # Gunakan semua core CPU
#     verbose = 2,    # Tampilkan progress
#     random_state = 42   # Pastikan hasil tetap reproducible
# )

# random_search.fit(X_train, y_train)

# best_model = random_search.best_estimator_


In [ ]:
# # Pilih model terbaik
# random_results = random_search.cv_results_
# random_results.keys()

# print("Best parameters based on recall:", random_search.best_params_)

In [ ]:
# # Cek metrik lainnya jika menggunakan multiple scoring
# print("Best model based on precision:", np.max(random_results['mean_test_precision']))
# print("Best model based on F1-score:", np.max(random_results['mean_test_f1']))

In [ ]:
# # Evaluasi Model Terbaik
# y_pred_best = best_model.predict(X_test)

# print("Classification Report:\n", classification_report(y_test, y_pred_best))

In [ ]:
# # Metrik tambahan
# accuracy = accuracy_score(y_test, y_pred_best)
# precision = precision_score(y_test, y_pred_best)
# recall = recall_score(y_test, y_pred_best)
# f1 = f1_score(y_test, y_pred_best)
# auc_score = roc_auc_score(y_test, y_pred_best)

# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1-Score: {f1:.4f}")
# print(f"AUC-ROC Score: {auc_score:.4f}")

### Halvingsearch

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {
    "model__n_estimators": [50, 100, 200, 500],
    "model__max_depth": [None, 10, 20, 30],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [1, 2, 5, 10],
    "model__max_features": ["sqrt", "log2", None],
    "model__bootstrap": [True, False],
    "model__class_weight": ["balanced", "balanced_subsample", None]
}

halving_search = HalvingGridSearchCV(
    base_model_pipe,    # Model pipeline
    param_grid,     # Gunakan param_grid yang sama
    factor = 3,     # Kandidat dikurangi 3x di setiap iterasi
    cv = 3,     # Cross-validation lebih kecil untuk kecepatan
    scoring = "recall",    # Single Metrik 
    refit = True, 
    n_jobs = -1,    # Gunakan semua core CPU
    verbose = 2,    # Tampilkan progress
    random_state = 42   # Untuk hasil yang konsisten
)

halving_search.fit(X_train, y_train)

best_model = halving_search.best_estimator_

# more than 1/2 hour

In [ ]:
# Pilih model terbaik berdasarkan akurasi
halving_results = halving_search.cv_results_
halving_results.keys()

print("Best parameters based on recall:", halving_search.best_params_)

In [ ]:
# Cek metrik lainnya jika menggunakan multiple scoring
print("Best model based on precision:", np.max(halving_results['mean_test_precision']))
print("Best model based on F1-score:", np.max(halving_results['mean_test_f1']))

In [ ]:
# Evaluasi Model Terbaik
y_pred_best = best_model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred_best))

In [ ]:
# Metrik tambahan
accuracy = accuracy_score(y_test, y_pred_best)
precision = precision_score(y_test, y_pred_best)
recall = recall_score(y_test, y_pred_best)
f1 = f1_score(y_test, y_pred_best)
auc_score = roc_auc_score(y_test, y_pred_best)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC Score: {auc_score:.4f}")

# Sample 2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE

# Simulasi data (gantilah dengan dataset asli)
X = pd.DataFrame(np.random.randn(226969, 10), columns=[f'feature_{i}' for i in range(10)])
y = np.array([0] * 223386 + [1] * 3583)  # Imbalanced dataset (fraud cases sangat sedikit)

# Stratified train-test split agar distribusi kelas tetap terjaga
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Transformasi
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=True, max_categories=50)
ordinal_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Kolom kategori (gantilah sesuai dataset Anda)
numeric_cols = [f'feature_{i}' for i in range(10)]  # Semua dianggap numerik
one_hot_cols = []  # Tambahkan jika ada fitur kategori one-hot
ordinal_cols = []  # Tambahkan jika ada fitur ordinal

# Column Transformer
prep_stage_2 = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numeric_cols),
        ("cat", categorical_transformer, one_hot_cols),
        ("ord", ordinal_transformer, ordinal_cols)
    ], remainder="passthrough"
)

# SMOTETomek untuk menangani data imbalanced
smote = SMOTE(sampling_strategy=0.8, random_state=42)
tomek = TomekLinks(sampling_strategy='majority')
sampling = SMOTETomek(smote=smote, tomek=tomek, random_state=42)

# Tangani kasus ZeroDivisionError jika kelas minoritas tidak ada di y_train
if np.sum(y_train == 1) == 0:
    scale_pos_weight = 1
else:
    scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)

# Pipeline dengan model XGBoost
xgb_pipe = Pipeline([
    ("preprocessor", prep_stage_2),
    ("sampling", sampling),
    ("model", XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42
    ))
])

# K-Fold Cross Validation dengan StratifiedKFold
k_values = [3, 5, 7, 10]
scores = []

for k in k_values:
    cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    score = np.mean(cross_val_score(xgb_pipe, X_train, y_train, cv=cv, scoring='accuracy'))
    scores.append((k, score))
    print(f"K = {k}: Mean Cross-Val Score = {score:.4f}")

# Menentukan nilai K terbaik
best_k = sorted(scores, key=lambda x: x[1], reverse=True)[0][0]
print(f"Optimal K untuk Cross-Validation: {best_k}")


# Sample 3

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from collections import Counter

# Oversampling dengan SMOTE (hanya meningkatkan minoritas hingga 30% dari mayoritas)
smote = SMOTE(sampling_strategy=0.3, random_state=42)

# Alternatif undersampling yang lebih hati-hati dengan NearMiss
undersample = NearMiss(version=3, n_neighbors=3)  # Mengurangi mayoritas tanpa kehilangan pola penting

# Lakukan resampling hanya dengan SMOTE terlebih dahulu
X_train_resample, y_train_resample = smote.fit_resample(X_train_tf, y_train)

# (Opsional) Jika ingin menggunakan undersampling setelah SMOTE
# X_train_resample, y_train_resample = undersample.fit_resample(X_train_resample, y_train_resample)

# Print distribusi sebelum & sesudah resampling
print("Before Resampling:")
print(y_train.value_counts(normalize=True) * 100)

print("\nAfter Resampling:")
print(y_train_resample.value_counts(normalize=True) * 100)

# Cek jumlah sampel setelah resampling
print(f"\nOriginal dataset shape: {Counter(y_train)}")
print(f"Resampled dataset shape: {Counter(y_train_resample)}")


# Sample 4

optuna (bayesian optimization)

In [ ]:
# import optuna
# from optuna.integration import SklearnPruningCallback
# from sklearn.model_selection import cross_val_score

# def objective(trial):
#     params = {}
    
#     if final_model_name == "Random Forest":
#         params = {
#             'n_estimators': trial.suggest_int('n_estimators', 100, 500),
#             'max_depth': trial.suggest_int('max_depth', 5, 30),
#             'min_samples_split': trial.suggest_int('min_samples_split', 2, 10)
#         }

#     elif final_model_name == "XGBoost":
#         params = {
#             'n_estimators': trial.suggest_int('n_estimators', 100, 500),
#             'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
#             'max_depth': trial.suggest_int('max_depth', 3, 10)
#         }

#     elif final_model_name == "LightGBM":
#         params = {
#             'num_leaves': trial.suggest_int('num_leaves', 31, 200),
#             'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
#             'n_estimators': trial.suggest_int('n_estimators', 100, 500)
#         }

#     elif final_model_name == "CatBoost":
#         params = {
#             'iterations': trial.suggest_int('iterations', 100, 500),
#             'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
#             'depth': trial.suggest_int('depth', 4, 10)
#         }

#     model = final_model.set_params(**params)
    
#     cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
#     scores = cross_val_score(model, X_train_mod, y_train_mod, cv=cv, scoring='roc_auc')
    
#     return np.mean(scores)

# # Optuna study
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=20, n_jobs=-1)

# best_params = study.best_params
# final_model = final_model.set_params(**best_params)

# print(f"\n🔍 Hyperparameter Terbaik ({final_model_name}): {best_params}")
# print(f"✅ Best ROC-AUC Score: {study.best_value:.3f}")


# Sample 5

In [ ]:
# search = HalvingGridSearchCV(
#     model, 
#     param_grid, 
#     factor=2,  # Setiap iterasi mengeliminasi setengah kandidat
#     scoring='roc_auc',
#     cv=5,  # Cross Validation 5-fold
#     verbose=1,
#     n_jobs=-1
# )

# search.fit(X_train_mod, y_train_mod)


# Sample 6

### Re-Sampling

In [ ]:
# # over-sampling
# smote = SMOTE(sampling_strategy = 0.8, 
#               k_neighbors = NearestNeighbors(n_jobs = -1), 
#               random_state = 42)

# # under-sampling
# tomek = TomekLinks(sampling_strategy = 'majority')

# # resampling
# sampling = SMOTETomek(smote = smote, 
#                       tomek = tomek, 
#                       random_state = 42)

# X_train_resample, y_train_resample = sampling.fit_resample(X_train_tf, y_train)

In [ ]:
# # over-sampling
# smote = SMOTE(sampling_strategy = 0.4, 
#               k_neighbors = NearestNeighbors(n_jobs = -1), 
#               random_state = 42)

# # under-sampling
# tomek = TomekLinks(sampling_strategy = 'auto')

# # resampling
# sampling = SMOTETomek(smote = smote, 
#                       tomek = tomek, 
#                       random_state = 42)

# X_train_resample, y_train_resample = sampling.fit_resample(X_train_tf, y_train)